
# Práctica 1 — **Embeddings & Búsqueda Semántica**  
**Sesión 4 · 2025-09-29**

**Objetivo:** Construir un *notebook* que:
1) Genere *embeddings* de un pequeño corpus,  
2) Cree un índice de búsqueda semántica,  
3) Permita realizar consultas,  
4) Evalue brevemente la relevancia de los resultados.

---
## Caso de uso: *Asistente de Conocimiento Interno (FAQ corporativa)*

Imaginemos que trabajas en una empresa de tecnología con varias áreas (TI, RR. HH., Legal, Datos). Los colaboradores realizan preguntas recurrentes: **políticas de vacaciones**, **acceso a VPN**, **solicitud de equipos**, **alta en plataformas cloud**, **soporte a BI**, etc.  
El objetivo es construir un prototipo de **búsqueda semántica** que encuentre las respuestas más relevantes a partir de un conjunto corto de **artículos/FAQs internos**.


## 1) Importaciones y utilidades

In [1]:

import os, re, math, json, random, unicodedata
from dataclasses import dataclass
from typing import List, Tuple, Dict

import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics.pairwise import cosine_similarity

random.seed(42)
np.random.seed(42)

def normalize_text(s: str) -> str:
    s = s.lower().strip()
    s = ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')
    s = re.sub(r'[^a-z0-9áéíóúñü\s]', ' ', s)
    s = re.sub(r'\s+', ' ', s).strip()
    return s

# Pequeña lista de stopwords ES (evitamos descargar NLTK)
STOPWORDS = set('''de la que el en y a los del se las por un para con no una su al lo como mas pero sus le ya o fue ha si porque muy sin sobre tambien me hasta hay donde quienes cual cuales cuando desde todo nos durante cada contra entre tras antes despues mi tus tus
esto esta estas estos ese esa esos esas aquel aquella aquellos aquellas soy eres es somos son estoy estas esta estamos estan
ser estar tener hace hacen hacer hacia podria puedes puede pueden podria deber debemos deben debo debo
ti tu usted ustedes nosotros nosotras ellos ellas su sus mio mia mios mias tuyo tuya tuyos tuyas suyo suya suyos suyas
mi mis nuestro nuestra nuestros nuestras vuestro vuestra vuestros vuestras
que quien quienes cual cuales cuanto cuanta cuantos cuantas donde adonde como cuando cuanto
'''.split())

def remove_stopwords(text: str) -> str:
    tokens = normalize_text(text).split()
    tokens = [t for t in tokens if t not in STOPWORDS and len(t) > 2]
    return ' '.join(tokens)


## 2) Dataset de FAQs internas (corpus de documentos)

In [ ]:

# Creamos un mini-corpus de artículos/FAQs internas (id, titulo, contenido)
docs = [
    (1, "Política de Vacaciones", """
    La empresa ofrece 12 días de vacaciones el primer año, con incremento gradual por antigüedad.
    Para solicitar vacaciones debes ingresar a la plataforma de RRHH con 15 días de anticipación.
    """),
    (2, "Acceso a VPN corporativa", """
    Para conectarte a la VPN necesitas un usuario activo, MFA y el cliente OpenVPN.
    Si tienes problemas, abre un ticket al área de TI incluyendo sistema operativo y ubicación.
    """),
    (3, "Solicitud de Equipo de Cómputo", """
    Los equipos (laptop/desktop) se solicitan a TI vía formulario interno.
    Se requiere aprobación del gerente y se asigna un activo con etiqueta y póliza de garantía.
    """),
    (4, "Alta en Plataformas Cloud (GCP/Azure/AWS)", """
    El alta en nubes públicas requiere curso de seguridad, aceptación de políticas y creación de cuentas con
    privilegios mínimos. Solicita acceso especificando proyecto y rol necesario.
    """),
    (5, "Soporte a Business Intelligence (BI)", """
    Para crear o actualizar dashboards en Looker/Power BI, envía un requerimiento con métricas,
    fuentes de datos y periodicidad. El equipo de Datos evalúa SLA y complejidad.
    """),
    (6, "Política de Home Office", """
    El trabajo remoto es híbrido: 3 días en oficina y 2 en casa. Las excepciones requieren autorización de RRHH.
    Se recomienda conexión estable y uso de VPN para recursos internos.
    """),
    (7, "Onboarding y Credenciales", """
    En tu primera semana recibirás credenciales de correo, gestor de contraseñas y acceso a gestor de identidades.
    Completa el curso de phishing y activa MFA en todas las aplicaciones.
    """),
    (8, "Procesos de Soporte TI", """
    Los tickets se abren en el portal de soporte. Prioridades: crítica, alta, media, baja.
    Incidentes críticos requieren puente de guerra y notificación a ciberseguridad.
    """),
    (9, "Política de Seguridad de la Información", """
    Todos los colaboradores deben seguir las guías de clasificación de datos, uso de contraseñas robustas y reporte
    de incidentes. El cifrado es obligatorio en equipos portátiles.
    """),
    (10, "Accesos a Bases de Datos", """
    Para acceder a bases de datos productivas se requiere justificación, ventana de mantenimiento y aprobación
    del dueño del dato. Se recomienda uso de cuentas de solo lectura.
    """),
]

df = pd.DataFrame(docs, columns=["id","titulo","contenido"])
df.head()


## 3) Limpieza y normalización

In [ ]:

df["texto_proc"] = df["contenido"].apply(remove_stopwords)
df[["id","titulo","texto_proc"]].head(10)


## 4) Embeddings con **TF‑IDF**

In [ ]:

tfidf = TfidfVectorizer(min_df=1, max_df=0.95, ngram_range=(1,2))
X_tfidf = tfidf.fit_transform(df["texto_proc"])
X_tfidf.shape


## 5) Reducción semántica con **LSA (SVD truncado)**

In [ ]:

n_components = 128 if X_tfidf.shape[1] >= 128 else max(2, min(64, X_tfidf.shape[1]-1))
svd = TruncatedSVD(n_components=n_components, random_state=42)
X_lsa = svd.fit_transform(X_tfidf)
X_lsa.shape



## 6) Embeddings* con **Autoencoder** (*red neuronal*)
Intentaremos entrenar un **autoencoder denso** sobre los vectores TF‑IDF para obtener una representación compacta.
> Si **no** tienes `tensorflow` instalado, el bloque fallará y el *notebook* continuará con LSA.


In [15]:
# ---------------------------------------------------------------
# Idea: entrenar una red neuronal (autoencoder) para comprimir los
# vectores TF-IDF a un espacio latente (embeddings) y luego usar
# ese espacio para búsqueda semántica.
#   • Entrada  : vector TF-IDF de cada documento
#   • Codificador (encoder): reduce dimensionalidad → "z"
#   • Decodificador (decoder): intenta reconstruir el TF-IDF original
#   • Pérdida : MSE entre entrada y salida (reconstrucción)
# Al terminar, usamos SOLO el "encoder" para obtener los embeddings.
# ===============================================================

AE_EMBED_DIM = 64  # ⇦ Dimensión del embedding latente "z". (Prueba 32 / 64 / 128)
EPOCHS = 40        # ⇦ Épocas de entrenamiento. (Más alto = más ajuste, riesgo de overfitting)
BATCH_SIZE = 8     # ⇦ Tamaño de batch. (Súbelo si tienes GPU/CPU potente)

# Pasamos de matriz dispersa (sparse) a densa (float32) para alimentar a Keras
X_dense = X_tfidf.toarray().astype("float32")

try:
    # Importamos TensorFlow/Keras dentro del try por si el entorno no lo tiene
    import tensorflow as tf
    from tensorflow import keras
    from tensorflow.keras import layers

    # Dimensión de entrada = # de términos/rasgos del TF-IDF
    input_dim = X_dense.shape[1]

    # ----- Definición del modelo -----
    # Capa de entrada: vector TF-IDF por documento
    inp = keras.Input(shape=(input_dim,))

    # Encoder: capas densas que van "apretando" la info
    x = layers.Dense(256, activation="relu")(inp)   # 1ª capa oculta (no linealidad)
    x = layers.Dense(128, activation="relu")(x)     # 2ª capa oculta

    # Bottleneck / Embedding: aquí vivimos en AE_EMBED_DIM dimensiones
    # activation=None → dejamos la representación "lineal" (puedes probar 'tanh')
    z = layers.Dense(AE_EMBED_DIM, activation=None, name="embedding")(x)

    # Decoder: intenta reconstruir el vector original desde "z"
    x = layers.Dense(128, activation="relu")(z)
    # Última capa del decoder: dimensión = input_dim.
    # activation="linear" porque queremos reconstruir valores continuos del TF-IDF
    out = layers.Dense(input_dim, activation="linear")(x)

    # Autoencoder completo: input → reconstruction
    autoenc = keras.Model(inp, out)

    # Optimizador y pérdida:
    #  - Adam(1e-3) suele converger bien en estos tamaños
    #  - MSE mide error de reconstrucción; también podrías probar MAE
    autoenc.compile(optimizer=keras.optimizers.Adam(1e-3), loss="mse")

    # ----- Entrenamiento -----
    # Entrenamos de forma no supervisada: entrada = objetivo
    history = autoenc.fit(
        X_dense, X_dense,
        epochs=EPOCHS,
        batch_size=BATCH_SIZE,
        verbose=0  # pon 1 si quieres ver la barra de progreso
    )

    # ----- Extraer el encoder para obtener embeddings -----
    # Creamos un modelo que va de la entrada "inp" al cuello de botella "z"
    encoder = keras.Model(inp, z)

    # Embeddings para TODO el corpus (uno por documento)
    X_ae = encoder.predict(X_dense, verbose=0)

    ae_available = True
    print("✅ Entrenado autoencoder. Forma de embeddings:", X_ae.shape)
    # Ejemplo de salida: (n_documentos, AE_EMBED_DIM)

except Exception as e:
    # Si no hay TensorFlow/Keras o falla algo, seguimos con LSA.
    ae_available = False
    X_ae = None
    print("⚠️ No se pudo usar TensorFlow/Keras. Continuaremos solo con LSA.")
    print("Detalle:", e)

# ---------------------------------------------------------------
# Notas y sugerencias:
# • Si ves sobre-ajuste (loss de entrenamiento muy baja pero mala
#   recuperación), baja EPOCHS, sube regularización (p.ej. layers.Dropout),
#   o reduce la capacidad (menos neuronas).
# • AE_EMBED_DIM muy chico puede perder info; muy grande puede memorizar.
# • Puedes probar activaciones 'tanh' en z si planeas usar similitud coseno.
# • Con GPU, puedes subir BATCH_SIZE para acelerar.
# ---------------------------------------------------------------

✅ Entrenado autoencoder. Forma de embeddings: (10, 64)


## 7) Índices de **búsqueda semántica**

In [18]:
# ---------------------------------------------------------------
# Creamos índices k-Nearest Neighbors (kNN) usando la métrica de
# distancia coseno (1 - similitud_coseno). Luego definimos:
#   • embed_query(): cómo convertir una consulta a vector según el método
#   • search(): recupera los top-k documentos más similares y arma una tabla
# ===============================================================

# Índices kNN con métrica coseno
# - Para TF-IDF y LSA siempre tenemos matrices listas.
# - Para AE solo si el autoencoder se entrenó (ae_available=True y X_ae no es None).
idx_tfidf = NearestNeighbors(n_neighbors=5, metric="cosine").fit(X_tfidf)
idx_lsa   = NearestNeighbors(n_neighbors=5, metric="cosine").fit(X_lsa)
idx_ae    = NearestNeighbors(n_neighbors=5, metric="cosine").fit(X_ae) if ae_available else None

def embed_query(query: str, method: str = "lsa"):
    """
    Convierte una consulta en su representación vectorial según el 'method'.

    Flujo:
      1) Limpia y normaliza la consulta (minúsculas, quita acentos, stopwords, etc.).
      2) Vectoriza con el mismo TF-IDF entrenado sobre el corpus.
      3) Proyecta:
         - 'tfidf' → se queda en el espacio TF-IDF.
         - 'lsa'   → aplica SVD entrenado (espacio semántico reducido).
         - 'ae'    → pasa por el encoder del autoencoder para obtener embeddings.
    """
    # Normalizamos y removemos stopwords para alinear con el preprocesamiento de los documentos
    q = remove_stopwords(query)

    # Vectorizamos la consulta con el mismo TF-IDF (¡importante usar el mismo vocabulario!)
    v_tfidf = tfidf.transform([q])

    if method == "tfidf":
        # Devuelve vector en el espacio TF-IDF (matriz dispersa)
        return v_tfidf
    elif method == "lsa":
        # Proyección al espacio latente de LSA (dimensionalidad reducida)
        return svd.transform(v_tfidf)
    elif method == "ae" and ae_available:
        # Pasamos a denso y obtenemos embeddings con el encoder del autoencoder
        return encoder.predict(v_tfidf.toarray().astype("float32"), verbose=0)
    else:
        # Errores comunes: pedir 'ae' cuando no se entrenó; o método mal escrito
        raise ValueError("Método no válido o AE no disponible")

def search(query: str, k: int = 3, method: str = "lsa") -> pd.DataFrame:
    """
    Realiza una búsqueda semántica:
      - Embebe la consulta con embed_query().
      - Busca sus vecinos más cercanos (top-k) en el índice correspondiente.
      - Devuelve un DataFrame con métricas y metadatos legibles.

    Columnas del resultado:
      • score(1-cos): distancia = 1 - similitud_coseno  → **MENOR es mejor** (0 = idéntico)
      • sim_cos     : similitud coseno                  → **MAYOR es mejor** (1 = idéntico)
      • id, titulo  : metadatos del documento recuperado
      • snippet     : fragmento del contenido para inspección rápida
    """
    # 1) Embedding de la consulta en el espacio correcto
    v = embed_query(query, method=method)

    # 2) Elegimos el índice según 'method' y recuperamos distancias e índices
    if method == "tfidf":
        dist, idxs = idx_tfidf.kneighbors(v, n_neighbors=k)
    elif method == "lsa":
        dist, idxs = idx_lsa.kneighbors(v, n_neighbors=k)
    elif method == "ae":
        # Si pidieron AE pero no hay índice (por no entrenar), esto daría error;
        # asumimos que 'embed_query' lo controló y solo llegamos aquí si AE está disponible.
        dist, idxs = idx_ae.kneighbors(v, n_neighbors=k)
    else:
        raise ValueError("Método de búsqueda desconocido")

    # 3) Construimos una tabla amigable con los resultados
    rows = []
    for d, i in zip(dist[0], idxs[0]):
        rows.append({
            "score(1-cos)": float(d),                 # 1 - coseno  → más bajo = mejor
            "sim_cos": float(1.0 - d),                # coseno      → más alto = mejor
            "id": int(df.iloc[i]["id"]),              # id del doc recuperado
            "titulo": df.iloc[i]["titulo"],           # título del doc
            "snippet": df.iloc[i]["contenido"].strip()[:180] + "..."  # vista previa
        })

    # 4) Regresamos un DataFrame ordenado como lo entrega kNN (mejor a peor)
    return pd.DataFrame(rows)

## 8) Consultas de ejemplo

In [ ]:
#---------------------------------------------------------------
# Objetivo: probar la búsqueda semántica con varias preguntas
# típicas del caso de uso "FAQ corporativa". Usaremos el método
# LSA (espacio semántico reducido con SVD) y pediremos el top-3.
# ===============================================================

queries = [
    # Cada elemento es una "consulta" del usuario final.
    # Procuramos cubrir varias áreas para ver si el buscador
    # trae el documento correcto por significado (no por palabras exactas).
    "¿Cómo pido vacaciones y cuántos días tengo?",                 # Esperado: doc de Vacaciones (id=1)
    "Necesito conectarme a la red interna desde casa",            # Esperado: VPN/teletrabajo (id=2)
    "Quiero acceso a bases de datos productivas",                 # Esperado: Accesos a BDs (id=10)
    "¿Quién me ayuda con dashboards en Looker o Power BI?",       # Esperado: Soporte BI (id=5)
    "Alta en cuentas de nube para un proyecto en GCP"             # Esperado: Alta en Cloud (id=4)
]

# k = 3  → Recupera los 3 documentos más similares (top-3).
# method = "lsa" → La consulta se vectoriza con TF-IDF y se proyecta al
#                  espacio semántico reducido por SVD (LSA).
# En este espacio comparamos por coseno: cuanto más cerca del 1, más similar.

for i, q in enumerate(queries, start=1):
    print(f"\n🔎 Consulta {i}: {q}")

    # La función search() devuelve un DataFrame con:
    # - 'score(1-cos)': distancia = 1 - coseno   → **MENOR es mejor** (0 = idéntico).
    # - 'sim_cos':      similitud coseno         → **MAYOR es mejor** (1 = idéntico).
    # - 'id':           id del documento recuperado.
    # - 'titulo':       título del documento.
    # - 'snippet':      primer fragmento del contenido para inspección rápida.
    res = search(q, k=3, method="lsa")

    # Recordatorio rápido para interpretar:
    print("   Nota: sim_cos cercano a 1 = más parecido; score(1-cos) cercano a 0 = mejor.")

    # Mostramos la tabla con los 3 mejores resultados según la similitud coseno.
    display(res)

## 9) Mini evaluación de relevancia (Precision@k y MRR)

In [ ]:

# ---------------------------------------------------------------
# Usamos un "golden set" muy simple: para cada consulta definimos
# cuál sería el ID del documento correcto (verdad terreno).
# Luego calculamos dos métricas:
#   • Precision@k: ¿aparece el doc correcto dentro del top-k?
#   • MRR (Mean Reciprocal Rank): 1/rank del doc correcto.
#      - Si el correcto está en 1er lugar → 1.0
#      - Si está en 2º → 0.5; en 3º → 0.333...; si no aparece → 0.0
# ===============================================================

# "golden" aproximado: mapeamos consulta -> id de documento esperado
gold = {
    # Nota: queries[i] viene de la celda anterior.
    # Este mapeo es solo para esta práctica (corpus pequeño).
    queries[0]: 1,   # Esperado: "Política de Vacaciones"
    queries[1]: 2,   # Esperado: "Acceso a VPN corporativa"
    queries[2]: 10,  # Esperado: "Accesos a Bases de Datos"
    queries[3]: 5,   # Esperado: "Soporte a Business Intelligence (BI)"
    queries[4]: 4,   # Esperado: "Alta en Plataformas Cloud (GCP/Azure/AWS)"
}

def precision_at_k(results_ids: List[int], relevant_id: int, k: int = 3) -> float:
    """
    Devuelve 1.0 si el documento relevante aparece dentro de los primeros k resultados,
    de lo contrario 0.0.
    - results_ids: lista de IDs ordenados por similitud (top primero).
    - relevant_id: ID del documento que consideramos la 'respuesta correcta'.
    - k: corte de la lista (top-k).
    """
    return 1.0 if relevant_id in results_ids[:k] else 0.0

def reciprocal_rank(results_ids: List[int], relevant_id: int) -> float:
    """
    Calcula el Reciprocal Rank (RR) para una consulta:
    - Busca en qué posición (rank) aparece el ID relevante.
    - Retorna 1/rank si se encuentra; 0.0 si no aparece en la lista.
    """
    for rank, doc_id in enumerate(results_ids, start=1):
        if doc_id == relevant_id:
            return 1.0 / rank
    return 0.0

def evaluate(method: str = "lsa", k: int = 3) -> dict:
    """
    Ejecuta la evaluación para todas las consultas del 'golden' usando
    el método de embeddings indicado (lsa, tfidf o ae si está disponible).

    Flujo:
    1) Para cada consulta q del golden:
       - Recupera top-k resultados con search(q, k, method).
       - Extrae la lista de IDs (ordenados por similitud).
       - Calcula Precision@k y RR para esa consulta.
    2) Promedia sobre todas las consultas → Precision@k promedio y MRR.

    Retorna:
      {"Precision@k": <promedio>, "MRR": <promedio>}
    """
    pks, mrrs = [], []
    for q, rel_id in gold.items():
        res = search(q, k=k, method=method)  # DataFrame con columnas: id, titulo, sim_cos, score(1-cos)...
        ids = res["id"].tolist()             # Extraemos solo los IDs en el orden retornado (mejor a peor).
        pks.append(precision_at_k(ids, rel_id, k=k))
        mrrs.append(reciprocal_rank(ids, rel_id))
    return {f"Precision@{k}": float(np.mean(pks)), "MRR": float(np.mean(mrrs))}

# ---------------------------------------------------------------
# Ejecutamos la evaluación con distintos métodos de embeddings.
# Interpreta así:
#  - Valores más altos son mejores (máx 1.0).
#  - Diferencias pequeñas en corpus chico son normales.
#  - En producción, usa más consultas y juicios humanos.
# ---------------------------------------------------------------
print("Resultados (LSA):", evaluate("lsa", k=3))
print("Resultados (TF-IDF):", evaluate("tfidf", k=3))
if ae_available:
    print("Resultados (Autoencoder):", evaluate("ae", k=3))



### Interpretación rápida
- **Precision@k** indica si el documento correcto aparece entre los *k* primeros resultados.
- **MRR** (Mean Reciprocal Rank) promedia la inversa del *rank* del documento relevante. Valores más altos ⇒ mejor.

> En un entorno real, conviene usar un conjunto de validación más grande, *judgements* manuales o heurísticos (por ejemplo, *click logs*), y comparar varios métodos (TF‑IDF / LSA / *embeddings* neuronales preentrenados).


## 10) Playground: escribe tus propias consultas

In [ ]:

# Escribe cualquier pregunta en lenguaje natural y recupera
# los 3 documentos más parecidos por significado usando LSA.
# Puedes cambiar:
#   • q: tu texto de consulta
#   • k: cuántos resultados quieres (top-k)
#   • method: "lsa" | "tfidf" | "ae" (si el autoencoder está disponible)
# ===============================================================

q = "Escribe aquí tu consulta libre, por ejemplo: necesito equipo de cómputo nuevo"

# Ejecutamos la búsqueda semántica.
# La función search() regresa un DataFrame con estas columnas:
#  - 'score(1-cos)': distancia = 1 - similitud coseno  → **MENOR es mejor** (0 = idéntico).
#  - 'sim_cos':      similitud coseno                   → **MAYOR es mejor** (1 = idéntico).
#  - 'id':           ID interno del documento.
#  - 'titulo':       título del documento recuperado.
#  - 'snippet':      fragmento del contenido para inspección rápida.
resultados = search(q, k=3, method="lsa")

# Mensaje recordatorio para interpretar correctamente las métricas
print("🔎 Consulta libre:", q)
print("   Nota: sim_cos cercano a 1 = más parecido; score(1-cos) cercano a 0 = mejor.")

# Mostramos la tabla con los top-k documentos
display(resultados)

# (Opcional) Vista ultra-resumida: títulos y similitud
print("\nResumen (título y similitud coseno):")
for _, row in resultados.iterrows():
    print(f" - [{row['id']:>2}] {row['titulo']}  |  sim_cos={row['sim_cos']:.3f}")



---
## 11) **Preguntas y ejercicios de comprensión**
1. **Explícalo con tus palabras:** ¿Qué diferencia hay entre los *embeddings* basados en **TF‑IDF**, **LSA** y  **Autoencoder**? ¿Cuál sería más robusto ante sinónimos?
2. **Experimenta con dimensiones:** Cambia el valor de `n_components` en la celda de LSA (por ejemplo 32, 64, 128). ¿Cómo varían **Precision@k** y **MRR**? (Escribe el codigo debajo marcado con Experimento 2)
3. **Red neuronal:** Modifica `AE_EMBED_DIM`, `EPOCHS` y `BATCH_SIZE` del autoencoder. ¿Mejora o empeora el desempeño? ¿Aumenta el *overfitting*?
(Escribe el codigo debajo marcado con Experimento 3)
4. **Amplía el corpus:** Agrega al *DataFrame* `df` al menos **5** documentos adicionales (por ejemplo, sobre *política de gastos*, *viajes*, *beneficios*). Re‑entrena y comenta los cambios en resultados.
5. **Análisis de relevancia:** Define **5 nuevas consultas** y una *golden truth* para cada una. Ejecuta la evaluación y reporta métricas con un breve comentario (2–3 líneas).

> **Entrega:** Sube este *notebook* con tus respuestas (en celdas Markdown) y las métricas obtenidas tras tus experimentos.
